# Tribolium embryo morphometry over time in Napari
Authors: Robert Haase, Daniela Vorkel, 2020

This is the pyclesperanto version of a workflow earlier [published for clij2](https://clij.github.io/clij2-docs/md/tribolium_morphometry/). 
[ImageJ Macro original](https://github.com/clij/clij2-docs/tree/master/src/main/macro/tribolium_morphometry.ijm)

This script is an example of heavy GPU-accelerated processing. It is recommended to use a dedicated
graphics card with at least 8 GB of GDDR6 memory. Otherwise, it may be quite slow.

Let's start by checking that pyclesperanto is installed and which GPU it uses.

In [1]:
"""
import pyclesperanto_prototype as cle
import numpy as np

# show all graphics cards
#print(cle._tier0._pycl.filter_devices())

# show only GPU devices
print(cle._tier0._pycl.filter_devices(dev_type='gpu'))

# selecting an Nvidia RTX
cle.select_device("Quadro M2200")
print("Using OpenCL device " + cle.get_device().name)
"""

'\nimport pyclesperanto_prototype as cle\nimport numpy as np\n\n# show all graphics cards\n#print(cle._tier0._pycl.filter_devices())\n\n# show only GPU devices\nprint(cle._tier0._pycl.filter_devices(dev_type=\'gpu\'))\n\n# selecting an Nvidia RTX\ncle.select_device("Quadro M2200")\nprint("Using OpenCL device " + cle.get_device().name)\n'

In [2]:
%gui qt

## Load a data set
The dataset shows a *Tribolium castaneum* embryo, imaged by a custom light sheet microscope, at a wavelength of 488nm (Imaging credits: Daniela Vorkel, Myers lab, MPI CBG). 
The data set has been resampled to a voxel size of 1x1x1 microns. The embryo expresses nuclei-GFP. We will use the dataset to detect nuclei and to generate an estimated cell-segmentation.

All processing steps are performed in 3D space.

In [6]:
from aicspylibczi import CziFile
import imgfile_tools as imf
from aicsimageio import AICSImage
from skimage import data
import napari
import dask
import dask.array as da
from IPython.display import display, HTML
from dask import delayed

filename = r"C:\Users\m1srh\OneDrive - Carl Zeiss AG\Testdata_Zeiss\LatticeLightSheet\LS_Mitosis_T=150-300.czi"

# get the metadata
md, addmd = imf.get_metadata(filename)
czi = CziFile(filename)

Detected Image Type (based on extension):  czi
Exception: 'ShortName'
Exception: 'ShortName'
Key not found: 'TubeLenses' Using Default Value = 1.0 for Tublens Magnification.
Key not found: 'TubeLenses' Using Default Value = 1.0 for Tublens Magnification.
Key not found: 'Name'
Key not found: 'Immersion'
Key not found: 'LensNA'
Key not found: 'TubeLenses' Using Default Value = 1.0 for Tublens Magnification.
Key not found: 'NominalMagnification' Using Default Value = 1.0 for Nominal Magnification.
Trying to extract Scene and Well information if existing ...
No valid Scene or Well information found: 'S'
Scales will not be rounded.


In [7]:
def load_image(czi, t=0):

    zstack = czi.read_image(S=0, T=t)
    
    return zstack

In [ ]:
lazy_imread = delayed(load_image)
reader = lazy_imread(czi, t=0)  # doesn't actually read the file
array = reader.compute()  # *now* it reads.

In [ ]:
"""
sample = imread(filenames[0])

lazy_imread = delayed(imread)  # lazy reader
lazy_arrays = [lazy_imread(fn) for fn in filenames]
dask_arrays = [
    da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays
]
# Stack into one large dask.array
stack = da.stack(dask_arrays, axis=0)
stack.shape  # (nfiles, nz, ny, nx)

# in jupyter notebook the repr of a dask stack provides a useful visual:
stack
"""

In [12]:
sp = [md['SizeC'], md['SizeZ'], md['SizeY'], md['SizeX']]

# create dask stack of lazy image readers
lazy_process_image = dask.delayed(load_image)  # lazy reader

lazy_arrays = [lazy_process_image(czi, t=t) for t in range(0, md['SizeT'])]

dask_arrays = [
    da.from_delayed(lazy_array, shape=sp, dtype=md['NumPy.dtype'])
    for lazy_array in lazy_arrays
]
# Stack into one large dask.array
dask_stack = da.stack(dask_arrays, axis=0)
print(dask_stack.shape)
dask_stack

(151, 2, 70, 413, 434)


,Array,Chunk
Bytes,7.58 GB,50.19 MB
Shape,"(151, 2, 70, 413, 434)","(1, 2, 70, 413, 434)"
Count,453 Tasks,151 Chunks
Type,uint16,numpy.ndarray


In [ ]:
#viewer = napari.Viewer()

In [14]:
# configure napari automatically based on metadata and show the OME-TIFF stack
layers = imf.show_napari(dask_stack, md,
                         blending='additive',
                         gamma=0.85,
                         add_mdtable=True,
                         rename_sliders=True)

Initializing Napari Viewer ...
Extract Channel as Dask.Array


ERROR:root:Unhandled exception:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\napari\_qt\event_loop.py", line 79, in gui_qt
    yield app
  File "C:\Users\m1srh\Documents\GitHub\czi_demos\imgfileutils.py", line 1335, in show_napari
    channel = array.compute().take(ch, axis=dimpos['C'])
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\dask\base.py", line 167, in compute
    (result,) = compute(self, traverse=False, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\dask\base.py", line 447, in compute
    results = schedule(dsk, keys, **kwargs)
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\dask\threaded.py", line 84, in get
    **kwargs
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\dask\local.py", line 486, in get_async
    raise_exception(exc, tb)
  File "C:\ProgramData\Anaconda3\envs\imageanalysis\lib\site-packages\dask\local.py"

In [ ]:
from napari.utils import nbscreenshot
nbscreenshot(viewer)